In [421]:
import torch
import torchvision
from torchvision import datasets
import torchvision.transforms as transforms
import torch.nn as nn
import shutil
import os
import time
import sys
from pathlib import Path
import functools
print = functools.partial(print, flush=True)
import numpy as np 


interest_num = [3,6]
img_size = 4
# number of subprocesses to use for data loading
num_workers = 0
# how many samples per batch to load
batch_size = 2
inference_batch_size = 1

print("="*100)
print("Demo 3 on MNIST. This script is for batch of data generation.")
print("\tStart at:",time.strftime("%m/%d/%Y %H:%M:%S"))
print("\tProblems and issues, please contact Dr. Weiwen Jiang (wjiang2@nd.edu)")
print("\tEnjoy and Good Luck!")
print("="*100)
print()

Demo 3 on MNIST. This script is for batch of data generation.
	Start at: 08/07/2020 17:42:08
	Problems and issues, please contact Dr. Weiwen Jiang (wjiang2@nd.edu)
	Enjoy and Good Luck!



In [422]:
def modify_target(target):
    for j in range(len(target)):
        for idx in range(len(interest_num)):
            if target[j] == interest_num[idx]:
                target[j] = idx
                break
    
    new_target = torch.zeros(target.shape[0],2)
        
    for i in range(target.shape[0]):        
        if target[i].item() == 0:            
            new_target[i] = torch.tensor([1,0]).clone()     
        else:
            new_target[i] = torch.tensor([0,1]).clone()
               
    return target,new_target

def select_num(dataset,interest_num):
    labels = dataset.targets #get labels
    labels = labels.numpy()
    idx = {}
    for num in interest_num:
        idx[num] = np.where(labels == num)
        
    fin_idx = idx[interest_num[0]]
    for i in range(1,len(interest_num)):           
        
        fin_idx = (np.concatenate((fin_idx[0],idx[interest_num[i]][0])),)
    
    fin_idx = fin_idx[0]    
    
    dataset.targets = labels[fin_idx]
    dataset.data = dataset.data[fin_idx]
    
    # print(dataset.targets.shape)
    
    dataset.targets,_ = modify_target(dataset.targets)
    # print(dataset.targets.shape)
    
    return dataset

def qc_input_trans(dataset):
    dataset.data = dataset.data
    return dataset


class ToQuantumData(object):
    def __call__(self, tensor):        
        data = tensor                
        input_vec = data.view(-1)
        vec_len = input_vec.size()[0]
        input_matrix = torch.zeros(vec_len,vec_len)
        input_matrix[0] = input_vec
        input_matrix = input_matrix.transpose(0,1)        
        u,s,v = np.linalg.svd(input_matrix)    
        output_matrix = torch.tensor(np.dot(u,v))            
        output_data = output_matrix[:,0].view(1,img_size,img_size)    
        return output_data
    

class ToQuantumMatrix(object):
    def __call__(self, tensor):        
        data = tensor                
        input_vec = data.view(-1)
        vec_len = input_vec.size()[0]
        input_matrix = torch.zeros(vec_len,vec_len)
        input_matrix[0] = input_vec
        input_matrix = np.float64(input_matrix.transpose(0,1))        
        u,s,v = np.linalg.svd(input_matrix)    
        output_matrix = torch.tensor(np.dot(u,v))                        
        return output_matrix


class ToQuantumData_Batch(object):
    def __call__(self, tensor):
        data = tensor
        input_vec = data.view(-1)
        vec_len = input_vec.size()[0]
        input_matrix = torch.zeros(vec_len,vec_len)
        input_matrix[0] = input_vec
        input_matrix = input_matrix.transpose(0,1)
        u,s,v = np.linalg.svd(input_matrix)
        output_matrix = torch.tensor(np.dot(u,v))
        output_data = output_matrix[:,0].view(data.shape)
        return output_data


# convert data to torch.FloatTensor
transform = transforms.Compose([transforms.Resize((img_size,img_size)),
                                transforms.ToTensor()])
# transform = transforms.Compose([transforms.Resize((img_size,img_size)),
#                                 transforms.ToTensor(),ToQuantumData()])
# transform = transforms.Compose([transforms.Resize((img_size,img_size)),transforms.ToTensor(),transforms.Normalize((0.1307,), (0.3081,))])
# choose the training and test datasets

# Path to MNIST Dataset
train_data = datasets.MNIST(root='../../pytorch/data', train=True,
                                   download=True, transform=transform)
test_data = datasets.MNIST(root='../../pytorch/data', train=False,
                                  download=True, transform=transform)

train_data = select_num(train_data,interest_num)
test_data =  select_num(test_data,interest_num)

# train_data = qc_input_trans(train_data)

# imshow(torchvision.utils.make_grid(train_data[0][0]))
# 
# sys.exit(0)

# prepare data loaders
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,
    num_workers=num_workers, shuffle=True, drop_last=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=inference_batch_size, 
    num_workers=num_workers, shuffle=True, drop_last=True)

def save_checkpoint(state, is_best, save_path, filename):
    filename = os.path.join(save_path, filename)
    torch.save(state, filename)
    if is_best:
        bestname = os.path.join(save_path, 'model_best.tar')
        shutil.copyfile(filename, bestname)


In [423]:
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
# functions to show an image
from matplotlib import cm


def imshow(img):
    img = img
    npimg = img.numpy()
    
    plt.imshow(np.transpose(npimg, (1, 2, 0)))    
    plt.show()
    
    image = np.asarray(npimg[0] * 255, np.uint8)    
    
    im = Image.fromarray(image,mode="L")
    im.save("32*32.jpg",cmap="gray") 
    im = im.resize((4,4),Image.BILINEAR)    
    
    plt.imshow(im,cmap='gray',)
    
    trans_to_tensor = transforms.ToTensor()
    trans_to_matrix = ToQuantumMatrix()
    plt.show()
    im.save("4*4.jpg",cmap="gray") 
    
    # print(trans_to_tensor(im))
    for row in trans_to_matrix(trans_to_tensor(im)).tolist():
        for num in row:
            print(num,end=",")
        print()

    
for batch_idx, (data, target) in enumerate(test_loader):
    torch.set_printoptions(threshold=sys.maxsize)
    # imshow(torchvision.utils.make_grid(data))
    print(data.shape)
    print(data,target)
    inputs = data.flatten()
    
    
    trans_to_tensor = transforms.ToTensor()
    trans_to_matrix = ToQuantumMatrix()
    
    
    # for row in trans_to_matrix((inputs)).tolist():
    #     for num in row:
    #         print(num,end=",")
    #     print()
    # 
    break

torch.Size([1, 1, 4, 4])
tensor([[[[0.0275, 0.1373, 0.0000, 0.0000],
          [0.0706, 0.1882, 0.1804, 0.0667],
          [0.0353, 0.3412, 0.2902, 0.1373],
          [0.0000, 0.0706, 0.0863, 0.0078]]]]) tensor([1])


In [424]:
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
# functions to show an image
from matplotlib import cm


def imshow(img):
    img = img
    npimg = img.numpy()
    
    plt.imshow(np.transpose(npimg, (1, 2, 0)))    
    plt.show()
    
    # image = np.asarray(npimg[0] * 255, np.uint8)    
    # 
    # im = Image.fromarray(image,mode="L")
    # im.save("32*32.jpg",cmap="gray") 
    # im = im.resize((4,4),Image.BILINEAR)    
    # 
    # plt.imshow(im,cmap='gray',)
    # 
    # trans_to_tensor = transforms.ToTensor()
    # trans_to_matrix = ToQuantumMatrix()
    # plt.show()
    # im.save("4*4.jpg",cmap="gray") 
    
    # print(trans_to_tensor(im))
    # for row in trans_to_matrix(trans_to_tensor(im)).tolist():
    #     for num in row:
    #         print(num,end=",")
    #     print()

#     
# for batch_idx, (data, target) in enumerate(test_loader):
#     torch.set_printoptions(threshold=sys.maxsize)
#     imshow(torchvision.utils.make_grid(data))
#     break




In [425]:
from lib_qc import * 
from lib_util import *
from lib_net import *

# Network Architecture: 2 layers and each layer contains 2 neurons

img_size = 4
device = torch.device("cpu")
layers = [2, 2]

model = Net(img_size,layers,True,[[1,1,1,1],[1,1]],True,False,False,False,True).to(device)

resume_path="checkpoint_0_0.6997.pth.tar"
print("=> loading checkpoint from '{}'<=".format(resume_path))
checkpoint = torch.load(resume_path, map_location=device)
epoch_init, acc = checkpoint["epoch"], checkpoint["acc"]
model.load_state_dict(checkpoint["state_dict"])


for name, para in model.named_parameters():
    if "fc" in name:
        print(name,binarize(para))
    else:
        print(name, para)


# for batch_idx, (data, target) in enumerate(test_loader):
#     torch.set_printoptions(threshold=sys.maxsize)
#     # imshow(torchvision.utils.make_grid(data))
#     # print(data)
#     
#     Q_InputMatrix = ToQuantumMatrix()(data.flatten())
#     break
# 
# print("="*10)
# print(Q_InputMatrix.shape)

=> loading checkpoint from 'checkpoint_0_0.6997.pth.tar'<=
fc0.weight tensor([[-1., -1., -1., -1., -1., -1., -1., -1., -1.,  1., -1.,  1., -1., -1.,
         -1., -1.],
        [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1., -1.,  1., -1.,  1.,  1.,
          1.,  1.]], grad_fn=<BinarizeFBackward>)
fc1.weight tensor([[ 1., -1.],
        [ 1.,  1.]], grad_fn=<BinarizeFBackward>)
qca1.x_running_rot Parameter containing:
tensor([0.4383, 0.5617])
qca1.x_l_0_5 Parameter containing:
tensor([1., 0.])
qca1.x_g_0_5 Parameter containing:
tensor([0., 1.])


In [426]:
from qiskit import QuantumRegister, QuantumCircuit, ClassicalRegister
from qiskit.extensions import XGate, UnitaryGate
import qiskit
from math import sqrt 
import math
import copy

def get_index_list(input,target):
    index_list = []
    try:
        beg_pos = 0
        while True:
            find_pos = input.index(target,beg_pos)
            index_list.append(find_pos)
            beg_pos = find_pos+1
    except Exception as exception:        
        pass    
    return index_list

def change_sign(sign,bin):
    affect_num = [bin]
    one_positions = []
    
    try:
        beg_pos = 0
        while True:
            find_pos = bin.index("1",beg_pos)
            one_positions.append(find_pos)
            beg_pos = find_pos+1
    except Exception as exception:
        # print("Not Found")
        pass
    
    for k,v in sign.items():
        change = True
        for pos in one_positions:
            if k[pos]=="0":                
                change = False
                break
        if change:
            sign[k] = -1*v
    

def find_start(affect_count_table,target_num):
    for k in list(affect_count_table.keys())[::-1]:
        if target_num<=k:
            return k


def recursive_change(direction,start_point,target_num,sign,affect_count_table,quantum_gates):
    
    if start_point == target_num:
        # print("recursive_change: STOP")
        return
    
    gap = int(math.fabs(start_point-target_num))    
    step = find_start(affect_count_table,gap)
    change_sign(sign,affect_count_table[step])
    quantum_gates.append(affect_count_table[step])
    
    if direction=="r": 
        # print("recursive_change: From",start_point,"Right(-):",step)
        start_point = start_point - step
        direction = "l"
        recursive_change(direction,start_point,target_num,sign,affect_count_table,quantum_gates)
        
    else:        
        # print("recursive_change: From",start_point,"Left(+):",step)
        start_point = start_point + step
        direction = "r"
        recursive_change(direction,start_point,target_num,sign,affect_count_table,quantum_gates)
        
    

def guarntee_upper_bound_algorithm(sign,target_num,total_len,digits):        
    flag = "0"+str(digits)+"b"
    pre_num = 0
    affect_count_table = {}
    quantum_gates = []
    for i in range(digits):
        cur_num = pre_num + pow(2,i)
        pre_num = cur_num
        binstr_cur_num = format(cur_num,flag) 
        affect_count_table[int(pow(2,binstr_cur_num.count("0")))] = binstr_cur_num   
        
    if target_num in affect_count_table.keys():
        quantum_gates.append(affect_count_table[target_num])
        change_sign(sign,affect_count_table[target_num])    
    else:
        direction = "r"
        start_point = find_start(affect_count_table,target_num)
        quantum_gates.append(affect_count_table[start_point])
        change_sign(sign,affect_count_table[start_point])
        recursive_change(direction,start_point,target_num,sign,affect_count_table,quantum_gates)
    
    return quantum_gates

def qf_map_extract_from_weight(weights):    
    # Find Z control gates according to weights
    w = (weights.detach().cpu().numpy())
    total_len = len(w)
    target_num = np.count_nonzero(w == -1)
    if target_num > total_len/2:
        w = w*-1
    target_num = np.count_nonzero(w == -1)    
    digits = int(math.log(total_len,2))
    flag = "0"+str(digits)+"b"
    max_num = int(math.pow(2,digits))
    sign = {}
    for i in range(max_num):        
        sign[format(i,flag)] = +1
    quantum_gates = guarntee_upper_bound_algorithm(sign,target_num,total_len,digits)
    
    # for k,v in sign.items():
    #     print(k,v)
    # print(w)
        
    # Build the mapping from weight to final negative num 
    fin_sign = list(sign.values())
    fin_weig = [int(x) for x in list(w)]
    
    # print(fin_sign)
    # print(fin_weig)
    sign_neg_index = []    
    try:
        beg_pos = 0
        while True:
            find_pos = fin_sign.index(-1,beg_pos)            
            qiskit_position = int(format(find_pos,flag)[::-1],2)                            
            sign_neg_index.append(qiskit_position)
            beg_pos = find_pos+1
    except Exception as exception:        
        pass    
    

    weight_neg_index = []
    try:
        beg_pos = 0
        while True:
            find_pos = fin_weig.index(-1,beg_pos)
            weight_neg_index.append(find_pos)
            beg_pos = find_pos+1
    except Exception as exception:        
        pass    
    map = {}
    for i in range(len(sign_neg_index)):
        map[sign_neg_index[i]] = weight_neg_index[i]
            
    # print(map)
    
    ret_index = list(range(len(fin_weig)))
    
    for k,v in map.items():
        old_val = ret_index[k] 
        ret_index[k] = v
        ret_index[v] = old_val
    

    return quantum_gates,ret_index
    
    
def extract_model(model):
    layer_prop = {}
    batch_adj_prop = {}
    indiv_adj_prop = {}
    for name, para in model.named_parameters():
        if "fc" in name:
            layer_id = int(name.split(".")[0].split("c")[1])
            layer_prop[layer_id] = [para.shape[1],para.shape[0],binarize(para)]            
        elif "qca" in name:            
            if "l_0_5" in name or "running_rot" in name:
                layer_id = int(name.split(".")[0].split("a")[1])
                layer_fun = name.split(".")[1]
                if layer_id not in batch_adj_prop.keys():
                    batch_adj_prop[layer_id] = {}
                batch_adj_prop[layer_id][layer_fun] = para
        else:            
            print(name, para)
    
    # print(layer_prop)
    # print(batch_adj_prop)

    # First layer
    first_layer_num = layer_prop[0][1]
    
    first_layer_input_q = int(math.log(layer_prop[0][0],2))
    first_layer_addition_q = max(first_layer_input_q-2,0)        
    first_layer_batch_q = 0    
    if 0 in batch_adj_prop.keys():
        first_layer_batch_q = 2
    
    first_layer_q = first_layer_num*(first_layer_input_q+first_layer_batch_q)
    
    # print(first_layer_q)
    
    # Second layer
    second_layer_num = layer_prop[1][1]
    second_layer_input_q = int(math.log(layer_prop[1][0],2))
    second_layer_addition_q = max(first_layer_input_q-2,0)        
    second_layer_batch_q = 0
    if 1 in batch_adj_prop.keys():
        second_layer_batch_q = 2

    second_layer_q = second_layer_num*(second_layer_input_q+second_layer_batch_q)
    # print(second_layer_q)
    
    return first_layer_q,second_layer_q,first_layer_input_q,layer_prop,batch_adj_prop,max(first_layer_addition_q,second_layer_addition_q)
    
# Main part

for batch_idx, (data, target) in enumerate(test_loader):
    torch.set_printoptions(threshold=sys.maxsize)
    # imshow(torchvision.utils.make_grid(data))
    # print(data,target)
    Q_InputMatrix = ToQuantumMatrix()(data.flatten())
    break
print(Q_InputMatrix.shape)



def q_map_neural_compute_body(circ,inputs,iq,inference_batch_size,log_batch_size):
        
    quantum_gates,ret_index = qf_map_extract_from_weight(nn_prop[0][2][0])
    # print(ret_index)
    # print(quantum_gates)
    # 
    expand_for_batch_index = copy.deepcopy(ret_index)
    for b in range(inference_batch_size-1):
        start = len(ret_index)*(b+1)
        new_batch_index = [x+start for x in ret_index]
        expand_for_batch_index += new_batch_index
    index = torch.LongTensor(expand_for_batch_index)    
    Input0 = copy.deepcopy(Q_InputMatrix)
    Input0 = Input0[index]
    # print("for debug comparison")
    # print(Q_InputMatrix[:,0])
    
    print(Input0[:,0])
    circ.append(UnitaryGate(Input0, label="Input0"), inputs[0:iq])
    qbits = inputs[log_batch_size:iq]
    
    for gate in quantum_gates:
        z_count = gate.count("1")
        # z_pos = get_index_list(gate,"1")
        z_pos = get_index_list(gate[::-1],"1")    
        if z_count==1:
            circ.z(qbits[z_pos[0]])
        elif z_count==2:
            circ.cz(qbits[z_pos[0]],qbits[z_pos[1]])
        elif z_count==3:
            qiskit_library.ccz(circ,qbits[z_pos[0]],qbits[z_pos[1]],qbits[z_pos[2]],aux_qr[0])
        elif z_count==4:
            qiskit_library.cccz(circ,qbits[z_pos[0]],qbits[z_pos[1]],qbits[z_pos[2]],qbits[z_pos[3]],aux_qr[0],aux_qr[1])
    
    
    
    # print(circ)
    # backend = Aer.get_backend('unitary_simulator')
    # job = execute(circ, backend)
    # result = job.result()
    #     
    # torch.set_printoptions(threshold=sys.maxsize)
    # np.set_printoptions(threshold=sys.maxsize)
    # state = result.get_unitary(circ, decimals=4)
    # print(state[:,0])
    # sys.exit(0)


def q_map_neural_compute_extract(circ,inputs,iq,outputs,log_batch_size):        
    qbits = inputs[0:iq-log_batch_size]
    # qbits = inputs[log_batch_size:iq]
    for q in qbits:
        circ.h(q)
    
    circ.barrier()
    
    for q in qbits:
        circ.x(q)
    
    
    digits = log_batch_size
    flag = "0"+str(digits)+"b"
    # qbits = inputs[0:log_batch_size]
    qbits = inputs[iq-log_batch_size:iq]
    if log_batch_size!=0:
        for i in range(int(math.pow(2,log_batch_size))):        
            binstr_cur_num = format(i,flag)
            
            for pos in range(len(binstr_cur_num)):            
                if binstr_cur_num[pos]=="0":                
                    circ.x(qbits[pos])
                    
            if digits==1:                            
                qiskit_library.cccccx(circ, inputs[0:iq], outputs[i],aux_qr)            
            elif digits==2:
                qiskit_library.ccccccx(circ, inputs[0:iq], outputs[i],aux_qr)
            
            
            for pos in range(len(binstr_cur_num)):                
                if binstr_cur_num[pos]=="0":                
                    circ.x(qbits[pos])
            circ.barrier()
            
    else:
        qiskit_library.ccccx(circ, inputs[0], inputs[1], inputs[2], inputs[3], outputs[0], aux_qr[0], aux_qr[1])
    
    circ.barrier()
    qbits = inputs[log_batch_size:iq]
    for q in qbits:    
        circ.x(q)
    

torch.Size([16, 16])


In [427]:
import qiskit_library

f,s,iq,nn_prop,bn_prop,aux = extract_model(model)


log_batch_size = int(math.log(inference_batch_size,2))

f = f+(log_batch_size)*nn_prop[0][1]
iq = iq+log_batch_size
aux = aux+inference_batch_size-1

s = (inference_batch_size)*s
s = inference_batch_size*2

# f=5
# aux=1


f_qr = QuantumRegister(f,"fLayer")
s_qr = QuantumRegister(s,"sLayer")
aux_qr = QuantumRegister(aux,"aux")
c = ClassicalRegister(s,"reg")



circ = QuantumCircuit(f_qr,s_qr,aux_qr,c)

# circ = QuantumCircuit(f_qr,aux_qr)

# Build Inputs and Computation
q_map_neural_compute_body(circ,f_qr[0:iq],iq,inference_batch_size,log_batch_size)
circ.barrier()
q_map_neural_compute_body(circ,f_qr[iq:iq*2],iq,inference_batch_size,log_batch_size)
circ.barrier()

# Build the extraction of results
q_map_neural_compute_extract(circ,f_qr[0:iq],iq,s_qr[0:inference_batch_size],log_batch_size)
circ.barrier()
q_map_neural_compute_extract(circ,f_qr[iq:2*iq],iq,s_qr[inference_batch_size:2*inference_batch_size],log_batch_size)
circ.barrier()

for i in range(s):
    circ.measure(s_qr[i],c[i])

print(circ)

tensor([0.0049, 0.1849, 0.2725, 0.0097, 0.0876, 0.3796, 0.2530, 0.0827, 0.0925,
        0.2384, 0.5596, 0.0341, 0.0049, 0.1655, 0.4623, 0.2190],
       dtype=torch.float64)
tensor([0.0049, 0.1849, 0.2725, 0.0097, 0.0876, 0.3796, 0.2530, 0.0827, 0.0925,
        0.2384, 0.5596, 0.0341, 0.0049, 0.1655, 0.4623, 0.2190],
       dtype=torch.float64)
             ┌──────────┐              ░                           ░ ┌───┐ ░ »
fLayer_0: |0>┤0         ├──■───────■───░───────────────────────────░─┤ H ├─░─»
             │          │  │       │   ░                           ░ ├───┤ ░ »
fLayer_1: |0>┤1         ├──■───────■───░───────────────────────────░─┤ H ├─░─»
             │  unitary │  │       │   ░                           ░ ├───┤ ░ »
fLayer_2: |0>┤2         ├──┼───■───┼───░───────────────────────────░─┤ H ├─░─»
             │          │  │   │   │   ░                           ░ ├───┤ ░ »
fLayer_3: |0>┤3         ├──┼───┼───┼───░───────────────────────────░─┤ H ├─░─»
             └────────

In [428]:
from qiskit_library import *
import torch
import numpy as np
from random import randrange
import qiskit as qk
from qiskit import Aer
from qiskit import execute
import math
import sys
import random
import time
from tqdm import tqdm

from qiskit import IBMQ
# IBMQ.delete_accounts()
IBMQ.save_account('62d0e14364f490e45b5b5e0f6eebdbc083270ffffb660c7054219b15c7ce99ab4aa3b321309c0a9d0c3fc20086baece1376297dcdb67c7b715f9de1e4fa79efb')
IBMQ.load_account()

num_c_reg=2

def analyze(counts):
    mycount = {}
    for i in range(num_c_reg):
        mycount[i] = 0
    for k,v in counts.items():
        bits = len(k) 
        for i in range(bits):            
            if k[bits-1-i] == "1":
                if i in mycount.keys():
                    mycount[i] += v
                else:
                    mycount[i] = v
    return mycount,bits

def fire_ibmq(circuit,shots,iter,Simulation = False, printable=True,backend_name='ibmq_essex'):
    if printable:
        print(circuit)
    
    count_set = []
    start = time.time()
    for it in range(iter):
        if not Simulation:
            provider = IBMQ.get_provider('ibm-q-academic')
            # ibm-q-academic backends: 
            #  5 qubits: ibmq_valencia
            # 20 qubits: ibmq_poughkeepsie, ibmq_johannesburg,ibmq_boeblingen, ibmq_20_tokyo
            # 53 qubits: ibmq_rochester
            
            # To get a specific qubit backend: 
            backend = provider.get_backend(backend_name)
        else:
            backend = Aer.get_backend('qasm_simulator')
        job_ibm_q = execute(circuit, backend, shots=shots)
        job_monitor(job_ibm_q)
        result_ibm_q = job_ibm_q.result()
        counts = result_ibm_q.get_counts()
        count_set.append(counts)
    end = time.time()
    print("Simulation time:", end - start)

    return count_set


print("="*50)
print("Start simulation:")
start = time.time()        
iters = 1
qc_shots=1000000
counts = fire_ibmq(circ,qc_shots,iters,True,False)
end = time.time()
qc_time = end - start

(mycount,bits) = analyze(counts[0])
for b in range(bits):
    print (b,float(mycount[b])/qc_shots)
    
print("From QC:",counts)
print("Simulation elasped time:",qc_time)



Start simulation:
Job Status: job has successfully run
Simulation time: 5.079354524612427
0 0.373811
1 0.374672
From QC: [{'00': 392048, '11': 140531, '10': 234141, '01': 233280}]
Simulation elasped time: 5.0824432373046875


In [429]:
# model.eval()
quantum_data = ToQuantumData_Batch()(data)
output = model(quantum_data, False)

print(output)

w = torch.tensor([ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1., -1.,  1., -1.,  1.,  1., 1.,  1.])
quantum_data=quantum_data.view(2,-1)
print("Input")
# print(quantum_data)
# weighted = (quantum_data*w)
# print((weighted[0].sum()/4).pow(2))
# print((weighted[1].sum()/4).pow(2))

# tensor([0.0108, 0.1053, 0.1674, 0.0432, 0.0162, 0.1809, 0.3646, 0.1053, 0.0729,
#         0.0702, 0.2674, 0.0216, 0.1242, 0.2107, 0.1809, 0.0648, 0.0000, 0.0297,
#         0.2053, 0.0243, 0.0135, 0.2620, 0.2296, 0.4321, 0.0486, 0.0189, 0.4699,
#         0.0108, 0.0054, 0.0891, 0.0999, 0.0972], dtype=torch.float64)
# tensor([0.0108, 0.1053, 0.1674, 0.0432, 0.0162, 0.1809, 0.3646, 0.1053, 0.0729,
#         0.0702, 0.2674, 0.0216, 0.1242, 0.2107, 0.1809, 0.0648, 0.0000, 0.0297,
#         0.2053, 0.0243, 0.0135, 0.2620, 0.2296, 0.4321, 0.0486, 0.0189, 0.4699,
#         0.0108, 0.0054, 0.0891, 0.0999, 0.0972], dtype=torch.float64)

# Q_InputMatrix = ToQuantumMatrix()(data.flatten())

# 
# print(Q_InputMatrix[:,0])
# Q_InputMatrix = ToQuantumMatrix()(data[0].flatten())
# print(Q_InputMatrix[:,0])
# Q_InputMatrix = ToQuantumMatrix()(data[1].flatten())
# print(Q_InputMatrix[:,0])
# 
# mydata = [0.0000, 0.2344, 0.0732, 0.0000, 0.0146, 0.4345, 0.3027, 0.0439, 0.0146,
#         0.5273, 0.5908, 0.1123, 0.0000, 0.0586, 0.1172, 0.0098, 0.0000, 0.0185, 0.3193, 0.0787, 0.0046, 0.2823, 0.4026, 0.0278, 0.0972,
#         0.6154, 0.4720, 0.0370, 0.0324, 0.1805, 0.0555, 0.0000]
# Q_InputMatrix = ToQuantumMatrix()(torch.tensor(mydata))
# print(Q_InputMatrix[:,0])



	 tensor([[0.0049, 0.1849, 0.2725, 0.0097, 0.0876, 0.3796, 0.2530, 0.0827, 0.0925,
         0.4623, 0.5596, 0.2190, 0.0049, 0.1655, 0.2384, 0.0341]])
	 tensor([[0.1782, 0.1782]], grad_fn=<PowBackward0>)
	 tensor([[0.1782, 0.1782]], grad_fn=<PowBackward0>)
	 tensor([[0.2929, 0.7071]], grad_fn=<DivBackward0>)
	 tensor([[0.6028, 0.3972]], grad_fn=<AddBackward0>)
tensor([[0.6028, 0.3972]], grad_fn=<AddBackward0>)
Input
tensor([[0.0049, 0.1849, 0.2725, 0.0097, 0.0876, 0.3796, 0.2530, 0.0827],
        [0.0925, 0.4623, 0.5596, 0.2190, 0.0049, 0.1655, 0.2384, 0.0341]])


RuntimeError: The size of tensor a (8) must match the size of tensor b (16) at non-singleton dimension 1

In [ ]:
# circ.append(UnitaryGate(Q_InputMatrix, label="iswap"), f_qr[iq:iq*2])

# quantum_gates,ret_index = qf_map_extract_from_weight(nn_prop[0][2][0])
# expand_for_batch_index = copy.deepcopy(ret_index)
# for b in range(inference_batch_size-1):
#     start = len(ret_index)*(b+1)
#     new_batch_index = [x+start for x in ret_index]
#     expand_for_batch_index += new_batch_index
# index = torch.LongTensor(expand_for_batch_index)
# Input0 = copy.deepcopy(Q_InputMatrix)
# Input0 = Input0[index]
# circ.append(UnitaryGate(Input0, label="Input0"), f_qr[0:iq])
# qbits = f_qr[inference_batch_size-1:iq]
# 
# 
# for gate in quantum_gates:
#     z_count = gate.count("1")    
#     z_pos = get_index_list(gate[::-1],"1")    
#     if z_count==1:
#         circ.z(qbits[z_pos[0]])
#     elif z_count==2:
#         circ.cz(qbits[z_pos[0]],qbits[z_pos[1]])
#     elif z_count==3:
#         qiskit_library.ccz(circ,qbits[z_pos[0]],qbits[z_pos[1]],qbits[z_pos[2]],aux_qr[0])
#     elif z_count==4:
#         qiskit_library.cccz(circ,qbits[z_pos[0]],qbits[z_pos[1]],qbits[z_pos[2]],qbits[z_pos[3]],aux_qr[0],aux_qr[1])
    

# quantum_gates,ret_index = qf_map_extract_from_weight(nn_prop[0][2][1])
# expand_for_batch_index = copy.deepcopy(ret_index)
# for b in range(inference_batch_size-1):
#     start = len(ret_index)*(b+1)
#     new_batch_index = [x+start for x in ret_index]
#     expand_for_batch_index += new_batch_index
# index = torch.LongTensor(expand_for_batch_index)
# Input1 = copy.deepcopy(Q_InputMatrix)
# Input1 = Input1[index]
# circ.append(UnitaryGate(Input1, label="Input1"), f_qr[iq:iq*2])
# qbits = f_qr[iq+inference_batch_size-1:iq*2]
# for gate in quantum_gates:
#     z_count = gate.count("1")
#     z_pos = get_index_list(gate[::-1],"1")
#     if z_count==1:
#         circ.z(qbits[z_pos[0]])
#     elif z_count==2:
#         circ.cz(qbits[z_pos[0]],qbits[z_pos[1]])
#     elif z_count==3:
#         qiskit_library.ccz(circ,qbits[z_pos[0]],qbits[z_pos[1]],qbits[z_pos[2]],aux_qr[0])
#     elif z_count==4:
#         qiskit_library.cccz(circ,qbits[z_pos[0]],qbits[z_pos[1]],qbits[z_pos[2]],qbits[z_pos[3]],aux_qr[0],aux_qr[1])
# 
# circ.barrier()

# 
# qbits = f_qr[inference_batch_size-1:iq]
# for q in qbits:
#     circ.h(q)
# 
# circ.barrier()
# 
# for q in qbits:
#     circ.x(q)
# 
# 
# digits = int(math.log(inference_batch_size,2))
# flag = "0"+str(digits)+"b"
# qbits = f_qr[0:inference_batch_size-1]
# if inference_batch_size!=1:
#     for i in range(inference_batch_size):        
#         binstr_cur_num = format(i,flag)
#         for pos in range(len(binstr_cur_num)):            
#             if binstr_cur_num[pos]=="0":                
#                 circ.x(qbits[pos])
#             if digits==1:                            
#                 qiskit_library.cccccx(circ, f_qr[0:iq], s_qr[i*2],aux_qr)
#             if binstr_cur_num[pos]=="0":                
#                 circ.x(qbits[pos])            
# else:
#     qiskit_library.ccccx(circ, f_qr[0], f_qr[1], f_qr[2], f_qr[3], s_qr[0], aux_qr[0], aux_qr[1])
#         
# # qbits = f_qr[inference_batch_size-1:iq]
# # for q in qbits:    
# #     circ.x(q)
# 
# print(circ)
# circ.barrier()
# print(circ)
# sys.exit(0)
# 
# 
# 
# 
# qbits = f_qr[iq+inference_batch_size-1:iq*2]
# for q in qbits:
#     circ.h(q)
#     circ.x(q)
# qiskit_library.ccccx(circ, f_qr[4], f_qr[5], f_qr[6], f_qr[7], s_qr[1], aux_qr[0], aux_qr[1])
# qbits = f_qr[iq+inference_batch_size-1:iq*2]
# for q in qbits:
#     circ.x(q)
# circ.barrier()
# 
# 
# circ.measure(s_qr[0],c[0])
# circ.measure(s_qr[1],c[1])
# print(circ)
# 
# sys.exit(0)

# 
# print(Q_InputMatrix.flatten())
# print("=="*10)
# print(Input0.flatten())
# print("=="*10)
# print(Input1.flatten())